# **Reakcja Bray'a - Liebhafskiego**

## **Agenda:**
1. Opis układu równań 
1. sprowadzenie do układu dwóch równań
1. implementacja metody Gaussa 
1. implementacja metody Runge-Kutta 
1. porównanie obu wyników dla obu metod 

In [1]:
x = 5 
print(x*5)

25


działa essa odnosimy sie tutaj do [[1](#ref1)]

## **Metody numeryczne**
Metody numeryczne używamy kiedy rozwiązanie analityczne jest trudne lub niemożliwe do uzyskania. Wtedy przybliżamy rozwiązanie korzystając z algorytmów przybliżających wartości funkcji w danym punkcie. W tym projekcie przedstawimy metodę 
Eulera oraz metodę Rungego-Kutty czwartego rzędu (RK4).

### **Błędy obcięcia**
Błąd lokalny to wielkość odchylenia w pojedyńczym kroku,
Błąd globalny to wielkość odchylenia w końcowym momencie. 

## **Metoda Eulera** [[2](#ref2)]
Jedną z najprostrzych metod numerycznych jest metoda eulera, która przybliża pochodną funkcji za pomocą różnic skończonych, jest to metoda pierwszego rzędu, tzn. błąd globalny jest rzędu $O(n)$. Zakładamy że zmiana w krótkim okresie czasu (h) jest liniowa

Weźmy pewne równanie różniczkowe:

$ \frac{dy}{dt}= f(y, t)$

Następnie możemy przybliżyć wartość w punkcie $ t_{0} + h $ korzystając z operatora różnic skończonych, weźmy początkową wartość $ y_{0}, t_{0}$:

$ y'(t_{0} + h) = \frac{y(t_{0} + h) - y_{0}}{h} + R = f(t_{0}+h,y)$, gdzie $ R $ to reszta którą pomijamy. Stąd otrzymujemy:

$y(t_{0} + h) = y_{0} + hf(t_{0},y_{0})$

Rekurencyjnie:

$ y_{n + 1} = y_{n} + hf(t_{n}, y_{n}) $ 

dodaj skąd reszta znika i implementacje  w python 


In [ ]:
def euler_method(f, t0, y0, h, steps, k):
    """
    f: funkcja przyjmująca argumenty t, C 
    t0: wartość początkowa argumentu t
    y0: wartość początkowa funkcji y
    h: wielkość kroku
    steps: ilość kroków
    k: parametr modelu 
    """
    t_values = [t0]
    y_values = [C0]
    for _ in range(steps):
        t_next = t_values[-1] + h
        y_next = y_values[-1] + h * f(t_values[-1], y_values[-1], k)
        t_values.append(t_next)
        y_values.append(C_next)
    return np.array(t_values), np.array(y_values)

### **Metoda Runge-Kutty (RK4)[[3](#ref3)]**
Jest to bardziej zaawansowana metoda. Wykorzystuje 4 kroki obliczeń w każdym kroku czasowym, zapewnia wysoką precyzję i stabilność. Liczymy w niej określoną ilość kroków pośrednich $ f(t_{*}, y_{*})$ dla $ t_{n} \leq t_{*} \leq  t_{n+1} $ oraz korzystamy z rozwinięcia funkcji w szereg taylora. 

Wyznaczmy 4 kroki pośrednie: 
$k_{1} = hf(x_n, y_n)$

$k_{2} = hf(x_{n} + \frac{1}{2}h, y_{n} + \frac{1}{2}k_1)$

$k_{3} = hf(x_n + \frac{1}{2}h, y_{n} + \frac{1}{2}k_2)$

$k_{4} = hf(x_{n} + h, y_{n} + k_{3})$

Końcowo otrzymujemy wzór na kolejną wartość:
$y_{n+1} = y_n + \frac{1}{6}h \left( k_1 + 2k_2 + 2k_3 + k_4 \right)$

### **Rozwiązanie modelu reakcji Bray'a - Liebhafskiego czterech zmiennych** 

Skorzystamy z wbudowanej metody Runge-Kutty w funkcję solve_ivp

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys

# NIE WIEM CZY PARAMETRY R SĄ ODPOWANIE 
def bray_lievhafski_model(t, y, R1, R2, R3, R4, R5, R6):
    """Model równań różniczkowych reakcji cos tam jodu
    :param t: czas ustawiany przez solver 
    :param y: wartości początkowe a później wektory stanu 
     U0: wartość początkowa Kwasu jodowego III HIO2
     V0: wart. począt. Anionu jodkowego 
     Z0: wart. począt. Jodu cząsteczkowego 
     W0: wart. począt. tlenu cząsteczkowego 02
    :param R1: szybkość tworzenia HIO₂ z IO₃⁻ i I⁻
    :param R2: to szybkość zużycia HIO₂ przez reakcję z I⁻
    :param R3: tempo autokatalitycznego tworzenia HIO₂
    :param R4: tempo rozpadu HIO₂
    :param R5: tempo powstawania I⁻ z I₂
    :param R6: tempo usuwania tlenu (O₂)
    """
    U, E, I, R = y  # rozpakowujemy listę
    N = S + E + I + R  # wielkość całej populacji
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I
    return [dSdt, dEdt, dIdt, dRdt]


def solve_equations(funct, y0, beta, sigma, gamma, N, t_span=None):
    if t_span is None:
        t_span = [0, 200]
    sol = solve_ivp(funct, t_span, y0, args=(beta, sigma, gamma),
                    dense_output=True)
    return sol


def main():
    # Ustawienia domyślne (typowe wartości dla modelu SEIR)
    default_N = 1000
    default_S0 = 995
    default_E0 = 3
    default_I0 = 2
    default_R0 = 0
    default_beta = 0.5
    default_sigma = 0.2  # 1/5 dni (średni czas inkubacji)
    default_gamma = 0.1  # 1/10 dni (średni czas wyzdrowienia)
    default_tmax = 100

    parser = argparse.ArgumentParser(description='Model SEIR - symulacja epidemii')

    parser.add_argument('-N', "--N", default=default_N, type=float, help='Całkowita populacja')
    parser.add_argument("-S0", "--S0", type=float, default=default_S0, help='Początkowa liczba podatnych')
    parser.add_argument("-E0", "--E0", type=float, default=default_E0, help='Początkowa liczba narażonych')
    parser.add_argument("-I0", "--I0", type=float, default=default_I0, help='Początkowa liczba zakaźnych')
    parser.add_argument("-R0", "--R0", type=float, default=default_R0, help='Początkowa liczba ozdrowiałych')
    parser.add_argument("-beta", "--beta", type=float, default=default_beta, help='Współczynnik infekcji')
    parser.add_argument("-sigma", "--sigma", type=float, default=default_sigma,
                        help='Współczynnik inkubacji (1/średni czas inkubacji)')
    parser.add_argument("-gamma", "--gamma", type=float, default=default_gamma,
                        help='Współczynnik wyzdrowień (1/średni czas infekcji)')

    parser.add_argument('--tmax', type=float, default=default_tmax,
                        help='Maksymalny czas symulacji w dniach (domyślnie: 100)')
    parser.add_argument('-output', '--output',  type=str, default='seir_plot.png',
                        help='Nazwa pliku wynikowego (domyślnie: seir_plot.png)')

    args = parser.parse_args()

    try:
        # Sprawdzenie czy którekolwiek z podanych wartości są ujemne
        if any(x < 0 for x in [args.N, args.S0, args.E0, args.I0, args.R0, args.beta, args.sigma, args.gamma]):
            raise ValueError("Parametry muszą być nieujemne")

        if not np.isclose(args.S0 + args.E0 + args.I0 + args.R0, args.N, rtol=0.01):
            print(f"Uwaga: Suma S0+E0+I0+R0 ({args.S0 + args.E0 + args.I0 + args.R0}) różni się od N ({args.N})")

        y0 = [args.S0, args.E0, args.I0, args.R0]
        solution = solve_equations(seir_model,
                                   y0,
                                   args.beta,
                                   args.sigma,
                                   args.gamma,
                                   args.N,
                                   [0, args.tmax])

        plt.figure(figsize=(10, 6))
        plt.plot(solution.t, solution.y[0], label='Podatni (S)')
        plt.plot(solution.t, solution.y[1], label='Narażeni (E)')
        plt.plot(solution.t, solution.y[2], label='Zarażeni (I)')
        plt.plot(solution.t, solution.y[3], label='Wyzdrowiali (R)')
        plt.xlabel('Czas (dni)')
        plt.ylabel('Liczba osób')
        plt.title('Model SEIR')
        plt.legend()
        plt.grid(True)
        plt.savefig(args.output)
        plt.show()

        print("Podsumowanie rozwiązania:")
        print(solution)

    except Exception as e:
        print(f'Wystąpił błąd: {e}')
        sys.exit(1)


if __name__ == "__main__":
    main()


### Bibliografia:
<a id="ref1"></a> 
[1].  Lawrence K. Forbes, Andrew P. Bassom, Courtney Quinn, A mathematical model of the Bray–Liebhafsky reaction, data dostępu: 26.12.2025 [link](https://royalsocietypublishing.org/rspa/article/480/2290/20230964/66721/A-mathematical-model-of-the-Bray-Liebhafsky)

<a id="ref2"></a>
[2]. Płociniczak Ł. "Skrypt do równań różniczkowych", 3.2. Euler's method, data dostępu:  [link](https://www.bing.com/search?qs=HS&pq=p%c5%82o&sk=CSYN1UAS9LS6AS5&sc=25-3&q=p%C5%82ociniczak+pwr&cvid=eaac520e4e03451fb99e306b7df2d9f8&gs_lcrp=EgRlZGdlKgkIABBFGDsY-QcyCQgAEEUYOxj5BzIGCAEQRRg7MgYIAhBFGDkyBggDEC4YQDIGCAQQLhhAMgYIBRAAGEAyBggGEEUYPTIGCAcQRRg9MgYICBBFGD3SAQgxNTc0ajBqOagCCLACAQ&FORM=ANAB01&PC=DCTS&ntref=1)

<a id="ref3"></a>
[3]. data dostępu: [link](https://sundnes.github.io/solving_odes_in_python/ode_book.pdf)